In [3]:
import pandas as pd
import numpy as np

In [5]:
# have to put the urls in chronological order if you want to specify stop date. only works 1 region at a time
url_lst = ['https://docs.google.com/spreadsheets/d/1dXOullM12TCE0hnsMY2LT7gWQ4H6FPBXHk-tSOhESSg/edit#gid=1642263273'
]

sheet_lst = []

counter = 0
for i in url_lst:
    if counter == 0:
        sheet_url = i
        url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url).reset_index(drop=True)
        sheet_lst.append(df)

    else:
        sheet_url = i
        url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url).reset_index(drop=True)
        sheet_lst.append(df[df.columns[5:]])
    counter += 1

if len(url_lst) > 1:
    df = pd.concat(sheet_lst, axis=1).T.reset_index().T
elif len(url_lst) == 1:
    df = df.T.reset_index().T

In [6]:
import gspread
from google.oauth2.service_account import Credentials

scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file("credentials.json", scopes=scope)

client = gspread.authorize(creds)

In [11]:
def fill_nan(df, sheet_name):
    """
    df: Google Sheet that has been converted to a DataFrame
    stop_date: Date to stop at as a str (M/DD/YYYY), empty string otherwise (''). Eg: If you want to stop at '4/15/2022', stop_date = '4/16/2022'
    output: dictionary indexed by state with values as a list of dates missing data
    """
    nan_okay = ['GA', 'IN', 'MI', 'CT', 'ND', 'VA', 'MA', 'ME', 'TX', 'VT', 'TN', 'PA', 'CA', 'PR', 'MN', 'WA', 'WI', 'IL', 'OH']
    filtered = df.loc[(~df[4].isin(nan_okay)) & (df[3] != np.nan) & (df[3].str.contains(',')) &
         (df[3].str.contains('District of Columbia') == False) & (df[3].str.contains('(state)') == False)].reset_index()
    first_row = pd.DataFrame(df.iloc[0]).T
    df = first_row.append(filtered, ignore_index=True)
    counties = df[3]
    state = df[4]
    missing_dict = {}
    counter_key = {0:'tstpos', 1:'pbpos', 2:'mort', 3:'pbmort'}
    col_counter = 6
    row_offset = 2
    state_counts = df.groupby(4).count()[3]
    
    sheet=client.open(sheet_name)
    sheet1=sheet.get_worksheet(0)

    out = []

    for i in df.loc[:,5:]:
        

        curr_col = df[i]
        # list of the indices with a NaN value
        null_idx = np.where(curr_col.isnull())[0]

        if len(null_idx > 0):
            for j in null_idx:
                a = sheet1.cell(j + row_offset, col_counter).value
                out.append(a)

        col_counter += 1

    return out
    

In [12]:
fill_nan(df, 'Copy of Southeast-East-Northeast COVID-19 by County (TEAM ENTRY) 05-30-2022 to present')

/opt/homebrew/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


APIError: {'code': 429, 'message': "Quota exceeded for quota metric 'Read requests' and limit 'Read requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:929882765613'.", 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'service': 'sheets.googleapis.com', 'quota_location': 'global', 'quota_limit': 'ReadRequestsPerMinutePerUser', 'quota_limit_value': '60', 'consumer': 'projects/929882765613', 'quota_metric': 'sheets.googleapis.com/read_requests'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quota#requesting_higher_quota'}]}]}

In [3]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file("credentials.json", scopes=scope)

client = gspread.authorize(creds)

sheet=client.open("Copy of Southeast-East-Northeast COVID-19 by County (TEAM ENTRY) 05-30-2022 to present")
sheet1=sheet.get_worksheet(0)

In [11]:
print(sheet1.row_count, sheet1.col_count)
#1000, 26

1067 88


In [18]:
sheet1.cell(1067, 4).value

'Unknown '

In [16]:
sheet1.update_cell(1067, 6, "HII")

{'spreadsheetId': '1WbprHxAm9Wgef0NZEhN8_qBI1PP1ui9w1gN117UbZFg',
 'updatedRange': "'05-30-22 to present'!F1067",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}